# 1. Data & Library

In [ ]:
!pip install bayesian-optimization
!pip install catboost

# 연산
import numpy as np
import pandas as pd

# 기타
import random
import os
import glob
import warnings
warnings.filterwarnings("ignore")

# 계층적 샘플링
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.multiclass import type_of_target
from sklearn.model_selection import StratifiedKFold

# 모델링
from bayes_opt import BayesianOptimization
from catboost import CatBoostRegressor

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11687 sha256=e9df75649ab474a444214c834a9c76f5644f94db79dc842557177debae44935e
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization
     |████████████████████████████████| 65.7MB 58kB/s 


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/DACON/태양광/재구조화/train_재구조화')
test_df = pd.read_csv('/content/drive/MyDrive/DACON/태양광/재구조화/test_재구조화')
print(f'train shape: {train_df.shape} \ntest shape:  {test_df.shape}')

train shape: (52176, 192) 
test shape:  (3888, 190)


# 2. Stratified Sampling 

* submission 제출 시에는 TARGET 1,2에 대해서 모두 계층적 샘플링을 진행
* 중복되는 부분을 제외하기 위해 편의상 TARGET 1에 대해서만 코드가 존재

## TARGET 1

In [ ]:
# TARGET을 기준으로 계층적 샘플링을 하기 위해
def target_scaler(x):
  for i in range(200):
    if (x>i) & (x<=i+1):
      return i
    elif x==0:
      return 0

train_df['TARGET_day7_cata']=train_df['TARGET_day7'].apply(target_scaler)
# train_df['TARGET_day7_cata'].unique()
# array([ 0,  7, 15, 23, 30, 36, 41, 45, 48, 49, 47, 43, 39, 33, 26, 19, 10,
#        3,  8, 12, 17, 20, 28, 29, 44, 11,  4, 16, 25, 37, 38,  1,  5,  6,
#       14, 40, 32,  9,  2, 27, 22, 35, 34, 18, 13, 24, 31, 50, 51, 46, 21,
#       52, 42, 53, 55, 56, 54, 57, 59, 58, 60, 61, 62, 64, 63, 67, 68, 66,
#       65, 69, 71, 70, 72, 75, 76, 74, 77, 78, 73, 80, 79, 82, 83, 81, 85,
#       86, 84, 90, 91, 88, 89, 87, 92, 93, 94, 95, 96, 97, 98, 99])

label_encoder = LabelEncoder()
train_df.TARGET_day7_cata = label_encoder.fit_transform(train_df.TARGET_day7_cata)
print(type_of_target(train_df.TARGET_day7_cata)) # type 확인

# 연도별 앙상블을 위해 3년 데이터를 3분할
train_df1=train_df.iloc[:48*365,:]
train_df2=train_df.iloc[48*365:365*48*2,:]
train_df3=train_df.iloc[365*48*2:,:]

stf=StratifiedKFold(n_splits=5, shuffle=False, random_state=42)

#마지막 fold를 사용, 사실 모든 fold를 활용해야 하지만 Bayesian Optimization과 LightGBM으로 qunatile loss를 metric으로 kFold를 구현하지 못해서 마지막 fold만 이용
for train_index1,valid_index1 in stf.split(train_df1, train_df1.TARGET_day7_cata):
  print(train_index1, valid_index1)

for train_index2,valid_index2 in stf.split(train_df2, train_df2.TARGET_day7_cata):
  print(train_index2, valid_index2)

for train_index3,valid_index3 in stf.split(train_df3, train_df3.TARGET_day7_cata):
  print(train_index3, valid_index3)

multiclass
[ 1997  2084  2089 ... 17517 17518 17519] [   0    1    2 ... 7077 7514 7558]
[    0     1     2 ... 17517 17518 17519] [1997 2084 2089 ... 8707 8769 9055]
[    0     1     2 ... 17517 17518 17519] [ 5402  5403  5680 ... 12353 12366 12416]
[    0     1     2 ... 17517 17518 17519] [ 7272  7752  7798 ... 15719 15766 15910]
[    0     1     2 ... 15719 15766 15910] [ 7800  7801  7945 ... 17517 17518 17519]
[ 2235  2322  2465 ... 17517 17518 17519] [   0    1    2 ... 6887 6933 6981]
[    0     1     2 ... 17517 17518 17519] [2235 2322 2465 ... 8844 8892 8940]
[    0     1     2 ... 17517 17518 17519] [ 4493  4877  5068 ... 12687 12943 12983]
[    0     1     2 ... 17517 17518 17519] [ 7422  7607  7656 ... 15957 16006 16153]
[    0     1     2 ... 15957 16006 16153] [ 7608  7609  7657 ... 17517 17518 17519]
[ 1611  1612  1702 ... 17133 17134 17135] [   0    1    2 ... 6411 6552 7367]
[    0     1     2 ... 17133 17134 17135] [1611 1612 1702 ... 8817 8867 9036]
[    0     1     

In [ ]:
train_df2 = train_df2.reset_index(drop=True)
train_df3 = train_df3.reset_index(drop=True)

del train_df1['TARGET_day7_cata']
del train_df2['TARGET_day7_cata']
del train_df3['TARGET_day7_cata']

X_train_1 = train_df1[train_df1.index.isin(train_index1)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_train_1 = train_df1[train_df1.index.isin(train_index1)]['TARGET_day7']
X_valid_1 = train_df1[train_df1.index.isin(valid_index1)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_valid_1 = train_df1[train_df1.index.isin(valid_index1)]['TARGET_day7']

X_train_2 = train_df2[train_df2.index.isin(train_index2)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_train_2 = train_df2[train_df2.index.isin(train_index2)]['TARGET_day7']
X_valid_2 = train_df2[train_df2.index.isin(valid_index2)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_valid_2 = train_df2[train_df2.index.isin(valid_index2)]['TARGET_day7']

X_train_3 = train_df3[train_df3.index.isin(train_index3)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_train_3 = train_df3[train_df3.index.isin(train_index3)]['TARGET_day7']
X_valid_3 = train_df3[train_df3.index.isin(valid_index3)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_valid_3 = train_df3[train_df3.index.isin(valid_index3)]['TARGET_day7']

print(X_train_1.shape, Y_train_1.shape, X_valid_1.shape, Y_valid_1.shape)
print(X_train_2.shape, Y_train_2.shape, X_valid_2.shape, Y_valid_2.shape)
print(X_train_3.shape, Y_train_3.shape, X_valid_3.shape, Y_valid_3.shape)

(14016, 190) (14016,) (3504, 190) (3504,)
(14016, 190) (14016,) (3504, 190) (3504,)
(13709, 190) (13709,) (3427, 190) (3427,)


# 3. Bayesian Optimization & CatBoost

* 제출 시에는 TARGET 1의 3년치 + TARGET 2의 3년치에 대해 모두 Bayesian Optimization & LightGBM 진행
* 중복을 제외하기 위해 TARGET 1의 첫 연도 데이터에 대해서만 코드 존재

## TARGET 1

#### year 1

In [ ]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5 ,0.6, 0.7, 0.8, 0.9]

for q in quantiles:
  def catboost_quantile_loss(X_train, y_train, X_valid, y_valid, loss_function='Quantile:alpha={}'.format(q),
                  iterations=500, learning_rate=0.05, depth=10, random_seed = 42,bagging_temperature = 0.2,
                  od_type='Iter', metric_period = 50, od_wait=20,min_data_in_leaf=32
                  ):
    
    
    catmodel = CatBoostRegressor( 
    loss_function= loss_function,
    iterations=iterations,
    learning_rate=learning_rate,
    depth=depth,
    random_seed=random_seed,
    bagging_temperature= bagging_temperature,
    od_type=od_type,
    metric_period=metric_period,
    min_data_in_leaf = min_data_in_leaf,        
    od_wait= 20,
    verbose = 100
    )

    catmodel.fit(X_train,
                 y_train,
                 eval_set=(X_valid,y_valid),
                 plot=False
            )

    predict = catmodel.predict(X_valid)
    
    best_quantile_loss_train = catmodel.best_score_['learn']['Quantile:alpha={}'.format(q)]
    best_quantile_loss_valid = catmodel.best_score_['validation']['Quantile:alpha={}'.format(q)]
        
        
    print('train_quantile_loss : {:.4f}, test_quantile_loss : {:.4f}'.format(best_quantile_loss_train, best_quantile_loss_valid))
    
    return - best_quantile_loss_valid

  def optimize_catboost(X_train,y_train,X_valid,y_valid, param_probe=None) :
    '''
    return the optimizer  and output the best hyperparameters found
    param_probe : dict of hyperparameters in order to guide the optimization
    '''
    def catboost_wrapper(learning_rate, depth, 
                         min_data_in_leaf, bagging_temperature,
                         loss_function = 'Quantile:alpha={}'.format(q), 
                         random_seed = 42,  iterations = 500) :
        '''
        wrapper for the function to optimize
        '''
        
        return catboost_quantile_loss(X_train=X_train, y_train=y_train ,
                             X_valid=X_valid ,y_valid=y_valid, 
                             iterations = int(iterations),
                             random_seed = random_seed, 
                             learning_rate=learning_rate, 
                             depth=int(depth),
                             min_data_in_leaf= int(min_data_in_leaf),
                             bagging_temperature = bagging_temperature
              )
    
    optimizer = BayesianOptimization(
        f = catboost_wrapper,
        pbounds= {
            "learning_rate" : (0.01,0.50),
            "depth" : (2,10),
            "min_data_in_leaf" : (1,10),
            "bagging_temperature" : (0,100),       
        },
        random_state = 42,
        verbose = 50,
    )
    if param_probe != None :
        optimizer.probe(
        params= param_probe,
        lazy=True,
        )
    
    optimizer.maximize(n_iter=5,
                      init_points=5)
    
    print("Final result:", optimizer.max)
    return optimizer

  print('-- Optimizing Quantile_{} Parameters --'.format(q))
  optimizer_cat = optimize_catboost(X_train=X_train_1, y_train=Y_train_1, 
                                  X_valid=X_valid_1, y_valid=Y_valid_1)
  
  params = optimizer_cat.max['params']
  params['depth'] = int(params['depth'])
  params['min_data_in_leaf'] = int(params['min_data_in_leaf'])
  
  cb_model = CatBoostRegressor(iterations=500,
                               random_seed = 42,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20,
                             loss_function='Quantile:alpha={0}'.format(q),
                             **params
                             )
  
  print('---- Predict',q,'----')
  cb_model.fit(X_train_1, Y_train_1,
             eval_set=(X_valid_1, Y_valid_1),
             use_best_model=True,
             verbose=100)
  
  pred_test_cat = cb_model.predict(test_df)
  pred_test_cat = pd.DataFrame(pred_test_cat)
  # pred_test_cat.to_csv('/content/drive/MyDrive/DACON/태양광/quantiles/catboost/year1/1_cat_quantile_{}.csv'.format(q), index=False)

-- Optimizing Quantile_0.1 Parameters --
|   iter    |  target   | baggin... |   depth   | learni... | min_da... |
-------------------------------------------------------------------------


0:	learn: 1.6029067	test: 1.6260920	best: 1.6260920 (0)	total: 255ms	remaining: 2m 7s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.361225134
bestIteration = 16

Shrink model to first 17 iterations.
train_quantile_loss : 1.6029, test_quantile_loss : 1.3612
|  1        | -1.361    |  37.45    |  9.606    |  0.3687   |  6.388    |


0:	learn: 1.7173248	test: 1.7202323	best: 1.7202323 (0)	total: 23.7ms	remaining: 11.8s
100:	learn: 1.3700803	test: 1.3354760	best: 1.3354760 (100)	total: 1.55s	remaining: 6.13s
200:	learn: 1.3283586	test: 1.3015825	best: 1.3010456 (192)	total: 2.99s	remaining: 4.44s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.300514416
bestIteration = 204

Shrink model to first 205 iterations.
train_quantile_loss : 1.3284, test_quantile_loss : 1.3005
|  2        | -1.301    |  15.6     |  3.248    |  0.03846  |  8.796    |


0:	learn: 1.7202991	test: 1.7222003	best: 1.7222003 (0)	total: 73.3ms	remaining: 36.6s
100:	learn: 1.3454507	test: 1.3584870	best: 1.3584870 (100)	total: 5.87s	remaining: 23.2s
200:	learn: 1.2469259	test: 1.2937330	best: 1.2927079 (193)	total: 11.4s	remaining: 17s
300:	learn: 1.1812260	test: 1.2835170	best: 1.2830474 (291)	total: 16.9s	remaining: 11.2s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.280918142
bestIteration = 313

Shrink model to first 314 iterations.
train_quantile_loss : 1.1812, test_quantile_loss : 1.2809
|  3        | -1.281    |  60.11    |  7.665    |  0.02009  |  9.729    |


0:	learn: 1.7014574	test: 1.7073318	best: 1.7073318 (0)	total: 20.8ms	remaining: 10.4s
100:	learn: 1.3111976	test: 1.2899045	best: 1.2899045 (100)	total: 1.52s	remaining: 6.02s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.282053641
bestIteration = 124

Shrink model to first 125 iterations.
train_quantile_loss : 1.3112, test_quantile_loss : 1.2821
|  4        | -1.282    |  83.24    |  3.699    |  0.09909  |  2.651    |


...

0:	learn: 5.7241677	test: 5.7256162	best: 5.7256162 (0)	total: 279ms	remaining: 2m 19s
100:	learn: 2.5610606	test: 2.7079222	best: 2.7079222 (100)	total: 27.6s	remaining: 1m 48s
200:	learn: 1.4319377	test: 1.5891401	best: 1.5891401 (200)	total: 55s	remaining: 1m 21s
300:	learn: 1.0061438	test: 1.1828175	best: 1.1828175 (300)	total: 1m 22s	remaining: 54.5s
400:	learn: 0.8345418	test: 1.0361795	best: 1.0361795 (400)	total: 1m 50s	remaining: 27.2s
499:	learn: 0.7595938	test: 0.9887862	best: 0.9887862 (499)	total: 2m 17s	remaining: 0us

bestTest = 0.9887861795
bestIteration = 499

train_quantile_loss : 0.7596, test_quantile_loss : 0.9888
|  7        | -0.9888   |  100.0    |  10.0     |  0.01     |  10.0     |


0:	learn: 5.1232093	test: 4.9868620	best: 4.9868620 (0)	total: 9.37ms	remaining: 4.68s
100:	learn: 1.0261614	test: 0.7382248	best: 0.7362209 (82)	total: 1.13s	remaining: 4.48s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.7362208751
bestIteration = 82

Shrink model to first 83 iterations.
train_quantile_loss : 1.0262, test_quantile_loss : 0.7362
|  8        | -0.7362   |  0.4478   |  2.323    |  0.2386   |  4.169    |


0:	learn: 3.8300634	test: 3.5446222	best: 3.5446222 (0)	total: 52.3ms	remaining: 26.1s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.8110161558
bestIteration = 19

Shrink model to first 20 iterations.
train_quantile_loss : 3.8301, test_quantile_loss : 0.8110
|  9        | -0.811    |  97.08    |  7.677    |  0.497    |  4.365    |


0:	learn: 3.7915666	test: 3.6378635	best: 3.6378635 (0)	total: 163ms	remaining: 1m 21s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.8575067168
bestIteration = 9

Shrink model to first 10 iterations.
train_quantile_loss : 3.7916, test_quantile_loss : 0.8575
|  10       | -0.8575   |  60.07    |  9.011    |  0.478    |  7.585    |
Final result: {'target': -0.631955598856875, 'params': {'bagging_temperature': 60.11150117432088, 'depth': 7.664580622368364, 'learning_rate': 0.020086402204943198, 'min_data_in_leaf': 9.72918866945795}}
---- Predict 0.9 ----


0:	learn: 5.6808026	test: 5.6752996	best: 5.6752996 (0)	total: 63.5ms	remaining: 31.7s
100:	learn: 1.5542670	test: 1.3209739	best: 1.3209739 (100)	total: 5.49s	remaining: 21.7s
200:	learn: 0.9727243	test: 0.7658020	best: 0.7658020 (200)	total: 11s	remaining: 16.4s
300:	learn: 0.8723774	test: 0.6741789	best: 0.6741789 (300)	total: 16.5s	remaining: 10.9s
400:	learn: 0.8218481	test: 0.6417428	best: 0.6417300 (398)	total: 22.2s	remaining: 5.47s
499:	learn: 0.7909929	test: 0.6320319	best: 0.6319556 (498)	total: 27.9s	remaining: 0us

bestTest = 0.6319555989
bestIteration = 498

Shrink model to first 499 iterations.


# 4. Submission

## TARGET 1 앙상블

In [ ]:
test_files_1  = sorted(glob.glob('/content/drive/MyDrive/DACON/태양광/quantiles/catboost/year1/*csv'))
test_files_2  = sorted(glob.glob('/content/drive/MyDrive/DACON/태양광/quantiles/catboost/year2/*csv'))
test_files_3  = sorted(glob.glob('/content/drive/MyDrive/DACON/태양광/quantiles/catboost/year3/*csv'))

In [ ]:
result_1 = []
result_1 = pd.DataFrame()

for i in range(0,len(test_files_1)):
  result_1 = pd.concat([result_1, pd.read_csv(test_files_1[i])], axis=1)

result_2 = []
result_2 = pd.DataFrame()

for i in range(0,len(test_files_2)):
  result_2 = pd.concat([result_2, pd.read_csv(test_files_2[i])], axis=1)

result_3 = []
result_3 = pd.DataFrame()

for i in range(0,len(test_files_3)):
  result_3 = pd.concat([result_3, pd.read_csv(test_files_3[i])], axis=1)

result_1.columns = ['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9']
result_2.columns = ['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9']
result_3.columns = ['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9']

print(result_1.shape, result_2.shape, result_3.shape)

(3888, 9) (3888, 9) (3888, 9)


In [ ]:
day_1_res = []
day_1_res = pd.DataFrame(columns=['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9'])

for i in result_1.columns:
  day_1_res[i] = (result_1[i] + result_2[i] + result_3[i]) / 3

def to_zero(x):
  if x < 0:
    return 0
  else:
    return x

for i in result_1.columns:
  day_1_res[i] = day_1_res[i].apply(to_zero)
  
display(day_1_res)

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,2.688103e-07,3.619210e-01,0.000000e+00,0.000000e+00,5.423489e-02,0.017449,6.904331e-02,4.604972e-07,0.045872
1,2.688103e-07,3.619208e-01,0.000000e+00,0.000000e+00,5.423489e-02,0.017449,6.904331e-02,4.604972e-07,0.045872
2,2.688103e-07,3.619208e-01,0.000000e+00,0.000000e+00,5.423489e-02,0.017449,6.904331e-02,4.604972e-07,0.045872
3,2.688103e-07,3.619208e-01,0.000000e+00,0.000000e+00,5.423489e-02,0.017449,6.904317e-02,4.604972e-07,0.045872
4,2.688103e-07,3.619208e-01,0.000000e+00,0.000000e+00,5.423489e-02,0.017449,6.904320e-02,4.604972e-07,0.045872
...,...,...,...,...,...,...,...,...,...
3883,9.839564e-07,2.037288e-07,3.122214e-07,3.919125e-07,3.637034e-08,0.019523,5.731814e-07,0.000000e+00,0.214571
3884,9.839564e-07,2.240480e-07,1.691648e-07,3.919125e-07,3.637034e-08,0.019523,5.731814e-07,0.000000e+00,0.214571
3885,9.839564e-07,0.000000e+00,3.133572e-07,3.911902e-07,3.637034e-08,0.019523,5.731814e-07,0.000000e+00,0.214571
3886,9.839564e-07,0.000000e+00,3.486931e-07,5.212565e-07,3.637034e-08,0.019523,5.599050e-07,0.000000e+00,0.214571


# 4. Permutation Importance

In [ ]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(cb_model,n_iter=5, random_state=42).fit(X_valid_1, Y_valid_1)
eli5.show_weights(perm, top=100, feature_names = X_valid_1.columns.tolist())

     |████████████████████████████████| 112kB 6.4MB/s 


Weight,Feature
0.1475 ± 0.0066,TARGET_day6
0.0178 ± 0.0012,TARGET_day3
0.0135 ± 0.0012,TARGET_day4
0.0073 ± 0.0012,TARGET_day2
0.0067 ± 0.0008,DHI_day6
0.0049 ± 0.0005,TARGET_day5
0.0047 ± 0.0005,TARGET_day0
0.0045 ± 0.0004,TARGET_day1
0.0041 ± 0.0004,Sun_DHI_per_day0
0.0032 ± 0.0010,DHI_day0
